### Installation

In [ ]:
%%capture
# Skip restarting message in Colab
#import sys; modules = list(sys.modules.keys())
#for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

In [ ]:
!nvidia-smi

In [1]:
import os
os.environ["VLLM_USE_V1"] = '0'

In [2]:
!pip install unsloth 
!pip install vllm==0.8.2
!pip install --upgrade pillow


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install bitsandbytes==0.45.5


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
#!wandb login


In [1]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"


In [2]:
#-------Change log info here---------
project = "Tourism"
datasetname = ""
TAGS = "GRPO"

steps = 5000
save_stepss = 1500
run_name=TAGS+"-"+model_name+"-steps-"+str(steps)
print("Total Steps to train: "+str(steps)+" save_stepss "+str(abs(save_stepss)))
#--------------------
import os
os.environ["WANDB_PROJECT"] = project  # name your W&B project
os.environ["WANDB_WATCH"] = "all"  # your W&B username
#os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # false default, log all model checkpoints
os.environ["WANDB_NOTES"] = "166K dataset, "+str(steps)+ " STEPS"
os.environ["WANDB_TAGS"] = TAGS # SFT or GRPO
saving_name= "moslehGen/"+model_name+"-Steps-"+str(steps)

Total Steps to train: 5000 save_stepss 1500


### Loading the model

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [3]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-21 10:25:45 [__init__.py:239] Automatically detected platform cuda.
Standard import failed for UnslothGKDTrainer: No module named 'UnslothGKDTrainer'. Using tempfile instead!


Load up `Llama 3.1 8B Instruct`, and set parameters

In [4]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3. vLLM: 0.8.2.
   \\   /|    NVIDIA RTX A5000. Num GPUs = 1. Max memory: 23.573 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/deepseek-r1-distill-llama-8b-unsloth-bnb-4bit with actual GPU utilization = 59.38%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.57 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.82 GB. Also swap space = 6 GB.
INFO 04-21 10:25:57 [config.py:585] This model supports multiple tasks: {'reward', 'embed', 'classify', 'score', 'generate'}. Defaulting to 'generate'.
WARNING 04-21 10:25:57 [

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-21 10:26:02 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-21 10:26:02 [model_runner.py:1146] Model loading took 5.7737 GB and 3.237600 seconds
INFO 04-21 10:26:04 [worker.py:267] Memory profiling takes 1.62 seconds
INFO 04-21 10:26:04 [worker.py:267] the current vLLM instance can use total_gpu_memory (23.57GiB) x gpu_memory_utilization (0.59) = 14.00GiB
INFO 04-21 10:26:04 [worker.py:267] model weights take 5.77GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 0.89GiB; the rest of the memory reserved for KV Cache is 7.28GiB.
INFO 04-21 10:26:04 [executor_base.py:111] # cuda blocks: 3728, # CPU blocks: 3072
INFO 04-21 10:26:04 [executor_base.py:116] Maximum concurrency for 512 tokens per request: 116.50x
INFO 04-21 10:26:08 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If o

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:19<00:00,  1.36it/s]

INFO 04-21 10:26:28 [model_runner.py:1570] Graph capturing finished in 20 secs, took 0.61 GiB
INFO 04-21 10:26:28 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 26.33 seconds



Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
<a name="Data"></a>



In [5]:
dataPath = "finetuning_finale.jsonl"

In [6]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    #data = load_dataset("openai/gsm8k", "main", split=split).select(range(1))
    data = load_dataset("json", data_files=dataPath, split="train")
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': x['answer']
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

In [7]:
dataset


Dataset({
    features: ['question', 'answer', 'prompt'],
    num_rows: 166021
})

In [ ]:
#dataset = dataset.rename_column('answer','Response')
#dataset = dataset.rename_column('question','Question')

#dataset.remove_columns(['Complex_CoT','answer'])

In [18]:
dataset

Dataset({
    features: ['question', 'answer', 'prompt'],
    num_rows: 166021
})

## Define Reward Function
We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [8]:
# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]


def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

<a name="Train"></a>
## Train the model

Now set up GRPO Trainer and all configurations!

In [9]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = steps,
    save_steps = save_stepss,
    max_grad_norm = 0.1,
    report_to="wandb", # Use this for WandB etc,
    run_name=run_name, # Use this for WandB etc,
    output_dir = "outputs",
)


Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 166,021 | Num Epochs = 1 | Total steps = 5,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 83,886,080/8,000,000,000 (1.05% trained)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mosleh (genpakt) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


-------------------- Question:
Wie bewerten deutsche Urlauber im Alter von 62 Jahren im Juni 2024 die Entwicklung in den Wohnvierteln im Mostviertel Österreichs kritisch während ihrer Reise? 
Answer:
Leider auch hier im Mostviertel in den Wohnviertel immer mehr Trend zu Schottergärten, Plastikzäunen und sonstigen Scheusslichkeiten, wie bei uns in Bayern leider auch. Da nag man mit dem Radl gar nicht mehr durchfahren. 
Response:
Okay, I need to figure out how German tourists aged 62 in June 2024 would critically assess the development in the wine villages in Austria's Mostviertel during their travels. Let me start by breaking down the query.

First, the target audience is German tourists who are 62 years old. They might be on a trip, possibly interested in wine, history, and culture. So, their critical assessment would likely focus on how well the wine villages are preserving their heritage while adapting to modernization.

I should consider the typical interests of such tourists. They 

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.031250,0.062500,200.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,200.000000,0.000641,0.000000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Was schätzen 80-jährige deutsche Urlaubsgäste im Februar 2022 besonders an ihrer langjährigen Verbundenheit mit der Region Bregenzerwald in Österreich? 
Answer:
Wir sind seit über 60 Jahren Besucher des Bregenzerwaldes und unsere Kinder mit Enkeln sind nun ebenfalls verwurzelt in die Region. 
Response:
Okay, so I need to figure out whether 80-year-old German holiday guests in February 2022 particularly valued their long-standing connection to the Bregenz Forest region in Austria. Hmm, let's break this down step by step.

First, I should consider the audience: 80-year-olds are typically not the prime demographic for traveling, especially in February. They might prefer more relaxed, less crowded places rather than popular tourist spots that are packed with people. But then again, some might have lifelong connections or family ties to the area.

The Bregenz Forest is known for its natural beauty, especially Viennese вален أقو Allen (Austria's Wachau Valley) 

<a name="Inference"></a>
## Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)

output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
## Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged(saving_name, tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged(saving_name, tokenizer, save_method = "merged_16bit", token = "YOUR_HF_TOKEN",)
        #model.push_to_hub_merged(saving_name, tokenizer, save_method = "lora", token = "")
# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged(saving_name, tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged(saving_name, tokenizer, save_method = "lora", token = "YOUR_HF_TOKEN")

In [ ]:
#if True: model.save_pretrained_merged(saving_name+"LORA", tokenizer, save_method = "lora",)
#if True: model.push_to_hub_merged(saving_name+"LORA", tokenizer, save_method = "lora", token = "YOUR_HF_TOKEN")

In [13]:
print("Hello")

Hello
